<a href="https://colab.research.google.com/github/ciccio1982/Colab/blob/main/ricezione_messaggio_solace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
pip install solace-pubsubplus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 4.1 MB/s eta 0:00:00


Before running the code, ensure you have the Solace PubSub+ Event Broker connection details ready. You'll need the host, message VPN name, and user credentials.

You can create a free Solace Cloud account and get these details if you don't have a broker instance.

In [ ]:
import solace.messaging as messaging
import logging
import threading
import time

# Configure logging for better visibility
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Solace Connection Parameters ---
SOLACE_HOST = "tcp://your_solace_broker_host:55555"  # e.g., "tcp://mr-abcdefghij.messaging.solace.cloud:55555"
SOLACE_VPN = "your_message_vpn"                # e.g., "default"
SOLACE_USERNAME = "your_username"             # e.g., "solace-cloud-client"
SOLACE_PASSWORD = "your_password"             # your client password
SOLACE_QUEUE_NAME = "my/test/queue"           # The queue you want to listen to

# Flag to control the consumer's execution loop
keep_running = True

Next, we'll define a `MessageReceiver` class that implements the `MessageHandler` interface. This class will contain the logic to process incoming messages.

In [4]:
class MessageReceiver(messaging.receiver.MessageReceiver):
    def __init__(self, service):
        self.service = service

    def on_message(self, message: messaging.receiver.InboundMessage):
        try:
            payload_str = message.get_payload_as_string()
            logging.info(f"Received message on topic '{message.get_destination_name()}' from queue '{message.get_destination_name()}' (queue destination name). Payload: {payload_str}")

            # Acknowledge the message to remove it from the queue
            message.ack_message()
            logging.info("Message acknowledged.")

        except Exception as e:
            logging.error(f"Error processing message: {e}")
            # Nack the message if processing failed, so it can be redelivered (if configured)
            message.nack_message()

    def on_rejected_message(self, message: messaging.receiver.InboundMessage):
        logging.warning(f"Rejected message received on topic '{message.get_destination_name()}'. This usually indicates a configuration error or message format issue. Payload: {message.get_payload_as_string()}")

    def on_message_receive_error(self, error_info: messaging.MessagingError):
        logging.error(f"Message receive error: {error_info}")


class ServiceEventHandler(messaging.MessagingServiceEventHandler):
    def on_service_event(self, event: messaging.ServiceEvent):
        logging.info(f"Solace Service Event: {event.get_event_type()} - {event.get_correlation_key()}")
        if event.get_event_type() == messaging.ServiceEvent.CONNECTION_LOST:
            logging.error("Connection to Solace broker lost!")
        elif event.get_event_type() == messaging.ServiceEvent.CONNECTION_DOWN_ERROR:
            logging.error("Connection to Solace broker went down with an error!")



NameError: name 'messaging' is not defined

Finally, we'll initialize the Solace messaging service, create a consumer, bind it to the specified queue, and start listening for messages. The consumer will run in a separate thread, and the main thread will wait for a stop signal.

In [5]:
def run_solace_consumer():
    global keep_running
    messaging_service = None
    consumer = None

    try:
        # 1. Build the Solace messaging service
        messaging_service = messaging.MessagingService.builder()
            .from_properties({
                messaging.SolaceProperties.Transport.HOST: SOLACE_HOST,
                messaging.SolaceProperties.Service.VPN_NAME: SOLACE_VPN,
                messaging.SolaceProperties.Authentication.SCHEME_BASIC_USER_NAME: SOLACE_USERNAME,
                messaging.SolaceProperties.Authentication.SCHEME_BASIC_PASSWORD: SOLACE_PASSWORD,
            })
            .with_service_event_handler(ServiceEventHandler())
            .build()

        # 2. Connect the messaging service
        logging.info("Connecting to Solace PubSub+ Event Broker...")
        messaging_service.connect()
        logging.info("Successfully connected to Solace PubSub+ Event Broker.")

        # 3. Create a queue destination
        queue = messaging.Queue.durable_queue(SOLACE_QUEUE_NAME)

        # 4. Create a persistent message consumer
        consumer = messaging_service.create_persistent_message_consumerBuilder()
            .with_queue(queue)
            .with_properties({
                messaging.receiver.PersistentMessageReceiverProperties.GENERATE_ACKNOWLEDGEMENT_FOR_SYNCHRONOUS_RECEIVE: False
            }) # Acknowledge messages manually
            .build()

        # 5. Register the message handler and start the consumer
        message_receiver = MessageReceiver(messaging_service)
        consumer.start(message_receiver)
        logging.info(f"Consumer started and listening on queue '{SOLACE_QUEUE_NAME}'.")
        logging.info("Press Ctrl+C or interrupt the kernel to stop the consumer.")

        # Keep the main thread alive while the consumer runs in the background
        while keep_running:
            time.sleep(1) # Sleep to prevent busy-waiting

    except messaging.MessagingError as e:
        logging.error(f"Solace Messaging Error: {e}")
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
    finally:
        if consumer:
            logging.info("Stopping consumer...")
            consumer.terminate()
            logging.info("Consumer terminated.")
        if messaging_service:
            logging.info("Disconnecting messaging service...")
            messaging_service.disconnect()
            logging.info("Messaging service disconnected.")
        logging.info("Solace consumer service stopped.")

# To start the consumer:
# run_solace_consumer()

# To stop the consumer from another cell or a different thread (e.g., if you wrap run_solace_consumer in a thread):
# keep_running = False


# You can run the consumer in a separate thread if you want to perform other tasks in the notebook concurrently.
# consumer_thread = threading.Thread(target=run_solace_consumer)
# consumer_thread.start()

# If you run it directly without a thread, it will block the notebook cell until it's stopped.
# To stop it, you can interrupt the kernel or set `keep_running = False` if running in a separate thread.

print("To start the consumer, uncomment and run `run_solace_consumer()` in a new cell. To stop it, interrupt the kernel.")

IndentationError: unexpected indent (ipython-input-1808245737.py, line 9)